In [23]:
import pickle
import gensim
import os
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
from source.preprocessing import Preprocessor, prep_tweets, prep_text

## Read data

In [3]:
handlers = ['sanchezcastejon', 'pablocasado_',
     'Pablo_Iglesias_', 'Santi_ABASCAL', 'Albert_Rivera']

In [4]:
data = {}
for handler in handlers:
    data[handler] = pd.read_csv('data/' + handler + '.csv')

## Preprocess data

In [5]:
p = Preprocessor(language='spanish', language_code='es', bad_list=['https', '://', 't', 'co'])

In [6]:
processed_data = {}
for handler in handlers:
    processed_data[handler] = data[handler].apply(lambda x: prep_tweets(x, p), axis=1)

In [7]:
processed_data[handlers[0]].head()

,tweet,created_at,id,processed_text,processed_tokens
0,El cruel y criminal zarpazo de la #ViolenciaDe...,Wed Dec 04 18:46:59 +0000 2019,1.202298e+18,cruel criminal zarpazo violenciadegénero dejar...,"[cruel, criminal, zarpazo, violenciadegénero, ..."
1,"Encuentro con el presidente turco,\n@RTErdogan...",Wed Dec 04 17:57:17 +0000 2019,1.202286e+18,encontrar presidente turco rterdogan agradecer...,"[encontrar, presidente, turco, rterdogan, agra..."
2,"En el nuevo escenario internacional, resulta v...",Wed Dec 04 16:07:44 +0000 2019,1.202258e+18,nuevo escenario internacional resultar vital d...,"[nuevo, escenario, internacional, resultar, vi..."
3,Todo lo que hemos conseguido hasta ahora se ba...,Wed Dec 04 15:45:16 +0000 2019,1.202253e+18,conseguir ahora basar capacidad nato adaptarse...,"[conseguir, ahora, basar, capacidad, nato, ada..."
4,"Conmemoramos 70 años de la @NATO, en los que l...",Wed Dec 04 15:26:04 +0000 2019,1.202248e+18,conmemorar año nato cohesión consenso ser mejo...,"[conmemorar, año, nato, cohesión, consenso, se..."


## Tfidf

In [8]:
texts = list(processed_data[handlers[0]]['processed_text'])

In [9]:
tf = TfidfVectorizer(min_df=1)

In [10]:
tf = tf.fit(texts)

In [11]:
tf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
trx_text = tf.transform(texts)

In [13]:
trx_text

<2276x9151 sparse matrix of type '<class 'numpy.float64'>'
	with 46987 stored elements in Compressed Sparse Row format>

In [14]:
centroids = normalize(trx_text, copy=False)

In [15]:
for i in centroids[0]:
    print(i)

  (0, 9009)	0.2909284224540783
  (0, 8550)	0.18218741505473188
  (0, 8359)	0.24140034844157018
  (0, 8328)	0.1621329864896666
  (0, 8071)	0.1277277909122243
  (0, 7645)	0.13863558969983786
  (0, 7498)	0.12861639859323745
  (0, 6874)	0.1941438979605356
  (0, 5823)	0.1972787767097214
  (0, 5629)	0.12861639859323745
  (0, 5202)	0.1868609595841857
  (0, 5134)	0.14389315693357835
  (0, 4888)	0.222627369728491
  (0, 4418)	0.2586298149509649
  (0, 3614)	0.18604115286238435
  (0, 2752)	0.30638456721172974
  (0, 2615)	0.21166106050217467
  (0, 2320)	0.15397498231926668
  (0, 2274)	0.12550992134294217
  (0, 2146)	0.21675121620934554
  (0, 2112)	0.2909284224540783
  (0, 2095)	0.21861107498651838
  (0, 1780)	0.18943179040975475
  (0, 1496)	0.1499936756874427
  (0, 986)	0.17252605869818227


In [48]:
query = 'pensiones'

In [49]:
query, _ = prep_text(query, p)
query_trans = tf.transform([query])
query_trans = normalize(query_trans, copy=False)

In [50]:
query_trans

<1x9151 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [51]:
D = linear_kernel(query_trans, centroids)
ind = np.argsort(D[0], axis=-1)[::-1]

In [52]:
ind

array([1116, 1521,  272, ..., 1523, 1524,    0], dtype=int64)

In [53]:
for i in ind:
    print(processed_data[handlers[0]]['tweet'].iloc[i] + '\n')

Las #pensiones son un derecho, no una mercancía. El PP pretende recortarlas hasta en un 40%. Lo que proponen es su privatización. El @PSOE va a garantizar las pensiones y blindarlas en la Constitución. Para mejorar la vida de la gente y construir así #LaEspañaQueQuieres /❤ https://t.co/SJefyks05u

Las #pensiones son para este Gobierno un derecho de ciudadanía, no una mercancía con la que hacer negocio. El #CMin aprueba la revalorización de más de 10 millones de pensiones, poniendo fin a un periodo de merma del poder adquisitivo de nuestros pensionistas. #GobiernoDelCambio https://t.co/QPoEdkxM2C

Miles de pensionistas se manifiestan hoy en Madrid pidiendo pensiones dignas. Su reivindicación es justa. Nuestro compromiso: garantizar su sostenibilidad,  revalorizarlas conforme al IPC real, blindar el sistema público de pensiones en la Constitución. #PENSIONISTASaMADRID16O https://t.co/vUhxe0MFqn

Sois prioridad de este #Gobierno. Trabajamos desde el primer día para defender las políticas 


La salud bucodental, en el SNS.
Nueva estrategia de salud mental.
Ingreso Mínimo Vital.
Pensiones dignas.
Pacto Nacional contra la pobreza infantil.
Ley contra la violencia infantil.

La España del Bienestar es nuestra mejor herramienta de cohesión social. #110compromisosPSOE /❤ https://t.co/ngXvvlDzbA

Nuestras condolencias y todo nuestro cariño para la familia y seres queridos del trabajador fallecido por una fuga de amoniaco hoy en el Polígono Norte de Tarragona, en La Pobla de Mafumet. Mis mejores deseos y toda la fuerza para la recuperación de los heridos.

Todo nuestro cariño para la familia y seres queridos del agente fallecido hoy en Cádiz en acto de servicio, y para sus compañeros/as de la @guardiacivil. Nuestro apoyo y agradecimiento a las FCSE, que arriesgan su vida cada día por nuestra seguridad y en la lucha contra el crimen

Comprender la esclerosis múltiple y conocer sus síntomas nos ayudará a ofrecer el mejor apoyo a quienes la sufren. Debemos avanzar en la visibilizac


Un equipo de investigación del @VHIO ha desarrollado un fármaco que actúa contra algunos de los tumores más agresivos. Ya se está probando con éxito en pacientes. 
Investigación y talento científico español para salvar vidas. ¡Enhorabuena, e infinitas gracias por vuestro trabajo! https://t.co/c1mrujexGa

80 años de la llegada a México de los primeros exiliados españoles a bordo del Sinaia. La fraternidad del pueblo mexicano y su presidente, Lázaro Cárdenas, dio la libertad y una patria a miles de personas que huían de la dictadura y la represión. GRACIAS. https://t.co/swU8YuHkC7

No hay mejores valores para una ciudad, un país o una sociedad que la #solidaridad y la #concordia. Felicidades a la ciudad polaca de Gdansk, que recibe el Premio Princesa de Asturias de la Concordia 2019 en reconocimiento a su defensa de las libertades y los derechos humanos. https://t.co/Imkrh7LsCW

¡GRANDES! Marc Gasol, Serge Ibaka y Scariolo hacen historia y logran el anillo de la #NBA con los Raptors. Ho


Ya en El Elíseo, con el presidente de la República Francesa, @EmmanuelMacron, para analizar los resultados de las elecciones al @Europarl_ES e intercambiar puntos de vista sobre la renovación de cargos en las instituciones europeas y la Agenda Estratégica de la #UE 2019-2024. https://t.co/7adUnFZkga

Orgullosos de nuestro gran Antonio Banderas, reconocido como el mejor actor en Cannes por su brillante interpretación en 'Dolor y Gloria'.
El actor malagueño es el quinto español en obtener esta distinción.

¡¡Enhorabuena, @antoniobanderas!! https://t.co/APSOUyFJCH

Un orgullo, una oportunidad y una enorme responsabilidad ser la 1ª delegación socialdemócrata en el @Europarl_ES. Trabajaremos por una alternativa progresista, socialdemócrata y de izquierdas a las políticas de austeridad. Una #Europa social, que proteja y redistribuya. #26MPSOE https://t.co/TjbBIpDNpA

Este domingo tienes la oportunidad de hacer que el cambio llegue a #Canarias, de que esta maravillosa tierra empiece a mirar 


Ahora que algunos van poniendo cordones sanitarios al @PSOE, que participó en la elaboración de nuestra carta magna, digamos alto y claro que la Constitución no es patrimonio de nadie, es la patria de todos los españoles y españolas. 
#SiempreHaciaDelante  ⏩ #VOTAPSOE 🌹 https://t.co/qgTrlkLMDs

Esta tarde estamos en #Albacete, con @garciapage. #SiempreHaciaDelante⏩

Nos vemos a partir de las 19h. Podéis seguirlo en directo, aquí: https://t.co/rXmVuzbomo

Porque Castilla-La Mancha es #MejorConPage. https://t.co/RnIyeCZhyq

La diferencia entre el @PSOE y la derecha y sus tres siglas está en cómo concebimos las sociedades. Nosotros no queremos sociedades que excluyan a quienes no piensan idéntico. Queremos sociedades integradoras. Por eso hay que votar socialista el #26M.
#SiempreHaciaDelante⏩ https://t.co/u1KQ9dDYqM

No hay derecho a esta derecha que sufrimos en España. El #26M, repitamos alto y claro el mensaje que no entendieron el #28A. España quiere mirar al futuro. Feminismo, convi

...


Hoy he presidido la reunión del Consejo de #Seguridad Nacional. Hemos aprobado las estrategias nacionales contra el terrorismo y el crimen organizado, así como el Plan Nacional de Biocustodia, que protegerá a #España contra el uso criminal de patógenos. https://t.co/ipw9I4qqsB https://t.co/AYAgcLQXPu

En unos minutos estaré junto a los compañeros y compañeras que asistirán al acto de presentación de la candidatura de @SerranoJAntonio a la alcaldía de #Murcia. 

Os espero. Podéis seguirlo en directo aquí ⤵ https://t.co/Cr9ykGoBDe

¡Viva España! ¡Claro! Pero no que viva enfrentada como quieren las tres derechas sino que viva unida a la igualdad entre españoles y a la diversidad territorial. Ser patriota es trabajar todos lo días porque en España se viva mejor.
#ElGobiernoNecesario https://t.co/RfwEAbbSTx

Muy pendiente de las noticias que nos llegan desde #Mali y del ataque terrorista a la misión de @UN_MINUSMA. Tragedias como esta nos hacen admirar aún más el esfuerzo y la dedicación d


La comunidad internacional debe permanecer unida ante cualquier forma de terrorismo. La fortaleza de la democracia y sus Gobiernos vencerá a la barbarie y al terror.
Mi solidaridad y apoyo al pueblo de #Colombia y a las familias de las víctimas y heridos. https://t.co/fRz2eB0XcB

Lanzarote, Tenerife, Zaragoza, Meco. La #ViolenciaMachista no da tregua. Cuatro mujeres han sido asesinadas en los últimos días en nuestro país. No permitiremos #NiUnPasoAtrás contra la #ViolenciaDeGénero. Todos y todas debemos volcarnos para terminar con este horror. https://t.co/uoEGO5FWGb

En unos días remitiremos a la Comisión Europea el Plan Nacional Integrado de Energía y Clima 2021-2030. Movilizará 235.000 mill.€ en los próximos 10 años para convertir el gran potencial de España en #renovables en una realidad.  #TransiciónEcológica 
https://t.co/hV97LTphLs https://t.co/Gmk0jJW6pQ

Debemos poner la ecología en el centro del futuro. Somos la primera generación que está padeciendo los efectos del #CambioC


Toda España se suma a la tristeza  infinita de la familia de Julen. Hemos seguido de cerca cada paso para llegar hasta él. Siempre agradeceremos el incansable esfuerzo de quienes lo buscaron durante todos estos días. Mi apoyo y cariño a sus padres y seres queridos.

Fortalecer la #ciencia y la #investigación es fortalecer nuestro país y su desarrollo. Debemos recuperar el talento, que científicos y científicas  vuelvan a casa. El #CMin aprueba 1450 plazas de la OEP destinadas a estabilización del personal investigador https://t.co/Ue2X2u25Cz https://t.co/8WZn2EP5u2

Ayer se cumplían 42 años del asesinato de los abogados de Atocha. Hoy el #CMin aprueba solicitar a Brasil la extradición activa de Carlos García Juliá. Terminaron con sus vidas pero no lograron parar la Democracia. Recordemos la historia para que nunca algo así vuelva a repetirse https://t.co/6unkcMKvrv

Hoy me he reunido con @IrmaAmaya, presidenta del @PARLACEN, al que pertenecen Guatemala, El Salvador, Honduras, Nicaragu


Pendientes de las informaciones que nos llegan desde El Campillo (Huelva). Todo mi apoyo y mi cariño para la familia y los seres queridos de Laura Luelmo.

España es reconocida como el emisor público de 2018. La prestigiosa publicación internacional #IFR destaca el crecimiento económico por encima de la media europea, la reducción de deuda nacional y su repercusión en la confianza de inversores. 
https://t.co/jqFaFumjTB

Cambios para avanzar. Este Gobierno ha presentado la modificación del artículo 49 de la Constitución y adaptarlo a las concepciones actuales. Como también hemos garantizado el derecho al sufragio de las personas con discapacidad. Reparamos así la justicia básica. #80AñosONCE https://t.co/ONJML6LbmJ

80 años de ilusión, lucha, solidaridad, fraternidad y cohesión.
@ONCE_oficial es ejemplo de la integración social, laboral, formación y empleo de las personas con distintas capacidades. 
¡Gracias por tanto cariño, y que la ilusión continúe! #80AñosONCE
https://t.co/inOotVJ


Después de 7 años de injusticia social los españoles no entenderán que la oposición vote en contra de unos presupuestos sociales que no dejan a nadie atrás.
Pido a la oposición, también a @PPopular y @CiudadanosCs, que se abstengan o voten a favor de #PGE2019. #PGEparaUnPaísMejor https://t.co/ZM2mQf8sJx

Comparto la angustia de la familia del pequeño de dos años que ha caído en un pozo en #Totalán #Malaga.
Todo mi apoyo a los padres y a los equipos de emergencias que están participando en el rescate. Mantengamos la esperanza en el mejor desenlace. https://t.co/z3CeGMfJET

"La voluntad de acuerdo no puede ser un recurso a utilizar según la coyuntura electoral. La voluntad de acuerdo es y debe seguir siendo un principio esencial de la vida pública". Certera y oportuna esta reflexión del ministro @abalosmeco. https://t.co/0eoWfYCbet

Que España siga avanzando en consolidación fiscal, cohesión social, dignidad laboral, creación de empleo y modernización económica. Ese es el objetivo de lo

# Semántico

In [59]:
tokens = list(processed_data[handlers[0]]['processed_tokens'])

In [ ]:
model_pretrained = KeyedVectors.load_word2vec_format('models/wiki.es.vec')

In [60]:
model = gensim.models.Word2Vec(
        tokens,
        size=300,
        sg=0, #Skip-Gram
        min_count=0,
        iter=100)

In [ ]:
model.build_vocab([list(model_pretrained.vocab.keys())], update=True)
model.intersect_word2vec_format(MODELS['pretrained'], lockf=1.0)

In [61]:
tf = TfidfVectorizer(vocabulary=model.wv.index2word)
tf = tf.fit(list(processed_data[handlers[0]]['processed_text']))

In [62]:
trx_text = tf.transform(texts) @ model.wv.syn0

In [64]:
trx_text.shape

(2276, 300)

In [65]:
centroids = normalize(trx_text, copy=False)

In [68]:
query = 'pensiones'

In [69]:
query, _ = prep_text(query, p)
query_trans = tf.transform([query]) @ model.wv.syn0
query_trans = normalize(query_trans, copy=False)

In [70]:
D = linear_kernel(query_trans, centroids)
ind = np.argsort(D[0], axis=-1)[::-1]

In [71]:
for i in ind:
    print(processed_data[handlers[0]]['tweet'].iloc[i] + '\n')

Blindaremos en la Constitución el carácter público del sistema de pensiones, su sostenibilidad financiera y su revalorización. Debemos garantizar un nivel de renta digno que evite el riesgo de pobreza de nuestros mayores.

Comparto mi entrevista con @65ymuchomas https://t.co/QWdl9WnMLO

Miles de pensionistas se manifiestan hoy en Madrid pidiendo pensiones dignas. Su reivindicación es justa. Nuestro compromiso: garantizar su sostenibilidad,  revalorizarlas conforme al IPC real, blindar el sistema público de pensiones en la Constitución. #PENSIONISTASaMADRID16O https://t.co/vUhxe0MFqn

Las #pensiones son un derecho, no una mercancía. Proponemos blindarlas en la Constitución para:

1. Garantizar su revalorización conforme al coste de la vida.
2. Defender su naturaleza pública.
3. Salvaguardar el sostenimiento financiero del sistema.

#HazQuePase /❤ #VOTAPSOE 🌹 https://t.co/ptp3n2PhVP

La revalorización de las #pensiones, la subida del SMI a 900 euros y el aumento del sueldo de los emplead


La administración electrónica debe avanzar al ritmo de la sociedad y adaptar su normativa.
Aprobamos un RDL para garantizar la seguridad pública, asegurando la prestación de servicios y un uso de la admón. legítimo que respete los derechos y libertades de la ciudadanía. #CMin https://t.co/zFA88ux5UF

✔Seguridad
✔La movilidad como un derecho y un elemento de cohesión social
✔Digitalización
✔Lucha contra la #emergenciaclimática, reduciendo emisiones y minimizando impacto del transporte.

Estos son los 4 ejes de la política de infraestructuras del Gobierno. #HolaGranada🚄 https://t.co/7MvZPTZEQK

Los #PGE2019 apuestan por la modernización de nuestra economía, la reconstrucción del Estado del Bienestar y la reducción del déficit público, con un objetivo central: las personas, especialmente en las más vulnerables.
Unos presupuestos para una #España mejor para todos y todas. https://t.co/V9ipMOg9nm

Los 33.793 nuevos puestos de #EmpleoPúblico aprobados hoy refuerzan la atención a la ciudadan


Cuando subimos el SMI a 900€ la derecha dijo que se destruiría empleo. La realidad es que hoy: 

Hay más afiliados a las seguridad social 
La afiliación de mujeres está en máximos históricos 
La contratación indefinida y de gente joven ha aumentado 
#HazQuePase /❤️  #VOTAPSOE 🌹 https://t.co/y9Ue0VE8mq

España necesita un sistema universitario potente y competitivo, y hoy escuchamos sus propuestas. Queremos impulsar una nueva ley que garantice su calidad, revertir los recortes e impulsar un nuevo sistema de becas y precios que facilite la equidad e igualdad de oportunidades. https://t.co/Zy33JxTMHB

Interesante conversación con @ajassy, CEO AWS @amazon, sobre habilidades digitales para jóvenes y mujeres, promoción de #StartUpNation o digitalización de la economía. Y un proyecto innovador: el desarrollo de política pública de servicios en la nube para la administración #WEF19 https://t.co/sb9g4X1XpD

Tras el 10N, Gobierno en diciembre.
Aprobaremos los PGE y una ley que acabe con la LOMC


Sois prioridad de este #Gobierno. Trabajamos desde el primer día para defender las políticas de atención a la dependencia y de pensiones públicas. Esa es la mejor forma de agradecer, en el #DíaInternacionalPersonasMayores, lo mucho que nos habéis aportado. ¡Gracias! https://t.co/nGkZ0gOftY

Es urgente frenar el drama de los desahucios. El Gobierno ya trabaja para que, en los casos de mayor vulnerabilidad, el procedimiento se paralice hasta que intervengan los Servicios Sociales. Así lo ha explicado hoy el ministro @albalosmeco en el Senado⤵️ https://t.co/K2qjVHrCh5 https://t.co/s525YDLPjm

Este Gobierno está comprometido con las trabajadoras y trabajadores #autónomos. Hoy he explicado en la #SesiónDeControl las medidas que vamos a poner en marcha para mejorar su situación laboral. https://t.co/JcsRUlJYxT

España es reconocida como el emisor público de 2018. La prestigiosa publicación internacional #IFR destaca el crecimiento económico por encima de la media europea, la reducción de de


España volverá a contribuir a la lucha global contra el #SIDA, la #malaria y la #tuberculosis. Porque la salud es un derecho de todos, defendemos la Cobertura Sanitaria Universal como objetivo global. Empecemos por combatir estas graves enfermedades en los países más vulnerables. https://t.co/jTiSOVpahF

#ProtegerEuropa para que #Europa proteja a sus ciudadanos y ciudadanas. Ese es nuestro objetivo y nuestra obligación. Para lograrlo propongo esta hoja de ruta con ocho objetivos de acción política: https://t.co/UpT2XCp1rh

Los resultados del Plan Director por un #TrabajoDigno ya son visibles. Nos propusimos combatir la temporalidad fraudulenta y la parcialidad abusiva, y no nos detendremos aquí. Este Gobierno seguirá trabajando por los #derechos laborales de todos y todas. https://t.co/GjEHXmu1bO https://t.co/NlMH14BvXN

Solo aumentará el progreso si crece la #IgualdadDeGénero. Trabajamos para acabar con la brecha salarial por género y hemos puesto en marcha diferentes acciones para q

...


En unos minutos estamos en #Toledo, con el presidente de #CastillaLaMancha, @garciapage, @Sergio_GP y la alcaldesa @milagrostolon. 
           Juntos haremos posible #LaEspañaQueQuieres /❤. 

🔴 En directo, a partir de las 19:00 horas:
https://t.co/NVFTWPaLLt

#HazQuePase /❤ https://t.co/yTi4CPGRl0

¡Dos oros y un bronce!
Felicidades a Lydia Valentín por sus tres medallas en los Mundiales de Halterofilia. Eres ejemplo de trabajo y superación, @lydikit. ¡Grande! 🥇🥇🥉 https://t.co/lxR82u6IwJ

En unos minutos arranca el cierre de campaña en #Madrid, donde estaré junto a @carmencalvo_ y @conJoseMFranco. 

¡Os esperamos!

Síguelo en directo a partir de las 17:45 horas: https://t.co/tDwqtGOMSE

#EstamosMuyCerca /❤️
#VOTAPSOE 🌹 https://t.co/LRaaq1Skf3

Nuevas generaciones que llegan con fuerza para hacer historia en nuestro deporte. ¡Hay mucho futuro 🏆🇪🇦!

¡Enhorabuena, campeones de Europa!  #U19EURO https://t.co/376wDxEyLw

Ya en Valencia para celebrar el #9deOctubre. Feliç dia a tots els val


Hace 15 años que el terror, la barbarie y la sinrazón golpeó a España. Fue el atentado más trágico  de nuestra historia. Hoy y siempre, sus víctimas, las que nos arrebataron y las que viven cada día con el dolor del recuerdo, están en nuestra memoria. No os olvidamos.
#Memoria11M https://t.co/vK0I9erCL2

Rendimos homenaje, en el cementerio de Colliure, a Antonio Machado. A él y a los miles de hombres y mujeres obligados a huir. Hoy les recordamos con orgullo.
Por un país para todos y todas. Nunca más una España en blanco y negro. #MemoriaDelExilio #80AniversarioExilioRepublicano https://t.co/54vJ0xJXPc

¡Ya estamos en #IllesBalears!
Gracias todos los que os habéis acercado a saludarnos. Siempre es un placer volver a encontrarse aquí con los compañeros y compañeras de @FSEPSOE.
Juntos por #LaEspañaQueQuieres  #VotaPSOE 🌹#HazQuePase /❤️ https://t.co/yJ5rgjekiA

Junto a la gran familia de socialdemócratas europeos, @TimmermansEU, @VDancila_PM, @antoniocostapm, @JosephMuscat_JM y Stefan L


Nos ha dejado Tomás Rodríguez Bolaños, alcalde durante 16 años de Valladolid. Municipalismo, transformación y compromiso es su legado. Porque las grandes políticas comienzan de abajo a arriba.
Un sentido abrazo a su familia y amigos.
https://t.co/oTvEHS8iJS

Quiero felicitar a @IvanDuque, elegido nuevo Presidente de Colombia. Mis mejores deseos para esta etapa que arranca en un momento crucial para el país. Son muchos los lazos que unen al pueblo español y al colombiano. Nuestra voluntad es seguir estrechándolos.

🇪🇸 90-78 🇵🇱 
 ¡Ya estamos en semis! Cada día más cerca. Los chicos de @BaloncestoESP se imponen ante Polonia en el #FIBAWC.

¡GRANDES!
#EspañaPolonia #SelMAS #SomosEquipo https://t.co/DJNEyDgpWg

Talento, humildad, esfuerzo y compromiso. Son los valores que transmite nuestra Selección Femenina de Baloncesto con su trabajo diario. Hoy las hemos despedido antes de su viaje al #EuroBasketWomen. Sois ejemplo y orgullo para toda la sociedad. ¡Suerte y éxitos! #SelFEM #ESPasión ht


Cerramos en #Valencia una campaña llena de ilusión y esperanza. De ganas para llenar nuestro país de futuro. Este domingo vamos a hacer que pase. ¡Juntos y juntas!

Arrancamos ya con @ximopuig, @abalosmeco y @SanGomezLopez: https://t.co/w7fKW0wKDr

#EstamosMuyCerca /❤ https://t.co/QVsgPBKi1Q

No hay palabras que puedan acompañar en este dolor. 

Mi sentido abrazo a Luis Enrique, y a toda la familia, ante la pérdida de su pequeña Xana. https://t.co/X05JBxSJq9

¡Una luchadora! Así es nuestra campeona olímpica y mundial, Carolina Marín. Un ejemplo de fortaleza y determinación que vuelve a tocar el ORO en el Open de China tras una dura lesión.

¡ENHORABUENA, Caro! 
#ESPasión 🇪🇦 @CarolinaMarin https://t.co/L1PJx93Rdt

Habéis luchado hasta el final, pero no ha sido posible. Un buen partido contra un rival fuerte. ¡España está orgullosa de vosotras, @SeFutbolFem! Habéis hecho historia y seguiréis haciéndonos disfrutar del fútbol. #JugarLucharyGanar #ESPUSA https://t.co/CwjDSTy9A5

Enhorabuen


Domingo de éxitos para España.
Tras ganar hoy en Le Mans, Fernando Alonso se hace con el Mundial de Resistencia. Y Marc Márquez consigue en Montmeló su cuarta victoria en siete carreras de #MotoGP. ¡Grandes! 🏆

¡IMPARABLE! Marc Márquez se corona por sexta vez campeón del mundo en #MotoGP en #ThaiGP. ¡Es su octavo título mundial! Todo un ejemplo de juventud, fortaleza y pasión. 

¡Enhorabuena, campeón 👏👏!  #8ball https://t.co/mnRKaj0iRR

Que la victoria de Alberto Fernández sea el triunfo y el impulso del pueblo argentino. Argentina contará siempre con mi apoyo y el del Gobierno de España. Nos une nuestro idioma, la Comunidad Iberoamericana y fuertes lazos que seguiremos estrechando.

Enhorabuena, @alferdez.

Nuestra historia y tradiciones nos unen y nos convierten en una sociedad plural, diversa y tolerante. Gracias @AytoViana, a su alcaldesa, Yolanda González, y a @mavichina, y felicidades al pueblo de #Viana en el VIII Centenario de su fundación. https://t.co/LvqJ9YH3iB

¡Enhorabuen